In [ ]:
## setup code for parsing arguments in my scripts
import argparse
from pathlib import Path
import sys

def main():
    parser = argparse.ArgumentParser(
        description="Extract phonation features from audio files."
    )
    parser.add_argument("input_path", type=Path, help="Path to audio file or folder")
    parser.add_argument("output_file", type=Path, help="Path to output feature file")
    parser.add_argument("static", type=str, help="Static features: 'true' or 'false'")
    parser.add_argument("plots", type=str, help="Plot results: 'true' or 'false'")
    parser.add_argument("format", type=str, choices=["csv", "txt", "npy", "kaldi", "torch"],
                        help="Output file format")

    args = parser.parse_args()

    input_path = args.input_path
    output_file = args.output_file
    static = args.static.lower() == 'true'
    plots = args.plots.lower() == 'true' 
    output_format = args.format

    # Input validation and handling
    if input_path.is_dir() or input_path.is_file():
        print(f"Processing {input_path}")
    else:
        print(f"Input path does not exist {input_path}")
        sys.exit(1)

    print(f"Output stored at {output_file}")
    print(f"Static {static}")
    print(f"Generate plots {plots}")
    print(f"Output format {output_format}")

if __name__ == "__main__":
    main()
